In [2]:
import pandas as pd
import requests

# URL del archivo Excel
url = "https://geoportal.dane.gov.co/descargas/divipola/DIVIPOLA_Municipios.xlsx"

# Hacer la solicitud HTTP para obtener el archivo
response = requests.get(url)
#print(response)

# Leer el archivo Excel en un DataFrame de pandas
#divipola_data = pd.read_excel(BytesIO(response.content))
divipola_data = pd.read_excel(response.content)

# Encontrar la primera fila que no es completamente NaN en la columna "Unnamed: 1"
start_row = divipola_data[divipola_data['Unnamed: 1'].notna()].index[0]

# Seleccionar esa fila como nombres de columna
new_columns = divipola_data.iloc[start_row]

# Asignar los nuevos nombres de columna
divipola_data.columns = new_columns

# Eliminar las filas anteriores, incluyendo la que se usó para los nombres de columna
divipola_data = divipola_data.iloc[start_row + 1:].copy()

# Restablecer el índice
divipola_data = divipola_data.reset_index(drop=True)

# Renombrar las columnas duplicadas agregando un prefijo
divipola_data.columns = [
    'CodigoDepartamento', 'NombreDepartamento', 'CodigoMunicipio', 'NombreMunicipio', 'Tipo: Municipio/Isla', 'Latitud', 'Longitud', 'Longitud'
]
# Encontrar la primera fila con NaN en la columna "Código"
nan_row_index = divipola_data[pd.isna(divipola_data['CodigoDepartamento'])].index[0]

# Eliminar todas las filas después de esa fila (incluyendo la fila con NaN)
divipola_data = divipola_data.iloc[:nan_row_index].copy()
divipola_data = divipola_data.drop(columns=divipola_data.columns[-1])

print(divipola_data.head(20).to_markdown(index=False, numalign="left", stralign="left"))
print(divipola_data.tail(20).to_markdown(index=False, numalign="left", stralign="left"))

| CodigoDepartamento   | NombreDepartamento   | CodigoMunicipio   | NombreMunicipio       | Tipo: Municipio/Isla   | Latitud     |
|:---------------------|:---------------------|:------------------|:----------------------|:-----------------------|:------------|
| 94                   | GUAINÍA              | 94883             | SAN FELIPE            | Área no municipalizada | 2.31218     |
| 94                   | GUAINÍA              | 94884             | PUERTO COLOMBIA       | Área no municipalizada | 2.58283     |
| 94                   | GUAINÍA              | 94885             | LA GUADALUPE          | Área no municipalizada | 1.40454     |
| 94                   | GUAINÍA              | 94886             | CACAHUAL              | Área no municipalizada | 3.38616     |
| 94                   | GUAINÍA              | 94887             | PANA PANA             | Área no municipalizada | 1.98106     |
| 94                   | GUAINÍA              | 94888             | MORICHAL       

C:\Users\adria\AppData\Local\Temp\ipykernel_26080\743117769.py:13: FutureWarning: Passing bytes to 'read_excel' is deprecated and will be removed in a future version. To read from a byte string, wrap it in a `BytesIO` object.
  divipola_data = pd.read_excel(response.content)


In [14]:
from pyspark.sql import SparkSession
from pyspark.sql import functions as F

# Crear una sesión de Spark
spark = SparkSession.builder.getOrCreate()

# Convertir el DataFrame de Pandas a Spark
divipola_spark_df = spark.createDataFrame(divipola_data)

divipola_spark_df.show()


Py4JJavaError: An error occurred while calling o305.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 6.0 failed 1 times, most recent failure: Lost task 0.0 in stage 6.0 (TID 6) (AdrianaC executor driver): org.apache.spark.SparkException: Python worker failed to connect back.
	at org.apache.spark.api.python.PythonWorkerFactory.createSimpleWorker(PythonWorkerFactory.scala:203)
	at org.apache.spark.api.python.PythonWorkerFactory.create(PythonWorkerFactory.scala:109)
	at org.apache.spark.SparkEnv.createPythonWorker(SparkEnv.scala:124)
	at org.apache.spark.api.python.BasePythonRunner.compute(PythonRunner.scala:174)
	at org.apache.spark.api.python.PythonRDD.compute(PythonRDD.scala:67)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.scala:367)
	at org.apache.spark.rdd.RDD.iterator(RDD.scala:331)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.spark.rdd.RDD.computeOrReadCheckpoint(RDD.